In [2]:
import json
import re
import time
from pathlib import Path
from tqdm.auto import tqdm

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# get replay website HTML
replays_url = "https://replay.pokemonshowdown.com/"
driver = webdriver.Chrome()
driver.get(replays_url)
text_box = driver.find_element(by=By.NAME, value="format")
submit_button = driver.find_elements(by=By.CSS_SELECTOR, value="body > div.pfx-panel > div > form:nth-child(5) > p > button")[0]
text_box.send_keys("gen8randombattle")
submit_button.click()
time.sleep(0.5)
html = driver.page_source
driver.close()

# make cache directory if none exists yet
path = Path("../cache/replays/")
path.mkdir(parents=True, exist_ok=True)

# cache logs and jsons
for m in tqdm(re.findall(r'href="(\/gen8randombattle-\d+)"', html)):
    url = f"https://replay.pokemonshowdown.com{m}"

    # cache .log
    path = Path(f"../cache/replays{m}.log")
    if not path.exists():
        log = requests.get(url + ".log").text
        with open(path, "x") as f:
            f.write(log)

    # cache .json
    path = Path(f"../cache/replays{m}.json")
    if not path.exists():
        response = requests.get(url + ".json")
        data = response.json()
        with open(path, "x") as f:
            json.dump(data, f)

 80%|████████  | 37/46 [00:49<00:12,  1.34s/it]


KeyboardInterrupt: 